<a href="https://colab.research.google.com/github/lmencisoe/ADL/blob/main/Project/Modelo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install shap
!pip install scikeras

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 569 kB 5.0 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [111]:
!pip install keras-tuner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 135 kB 5.0 MB/s 
     |████████████████████████████████| 1.6 MB 40.2 MB/s 


In [112]:
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')
import xgboost as xgb
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.decomposition import PCA
from pandas_profiling import ProfileReport
import tensorflow as tf
from sklearn.feature_extraction import DictVectorizer
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from keras.models import Sequential
import keras_tuner as kt
from xgboost import plot_importance
from keras.layers import Dropout, Flatten, Dense, Input, Rescaling, Lambda, MaxPooling2D, Conv2D, AveragePooling2D
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.compose import make_column_selector, ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, StandardScaler
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import ConfusionMatrixDisplay, precision_score, recall_score, f1_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.model_selection  import GridSearchCV
import tensorflow as tf
from scikeras.wrappers import KerasClassifier

from keras.models import Sequential
from keras.layers import Dense, Input
from keras.utils.vis_utils import plot_model

import statistics
from matplotlib import pyplot
import seaborn as sns
import shap
import warnings
import re
warnings.filterwarnings('ignore')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
new_df3 = pd.read_csv("/content/drive/MyDrive/Proyecto_ADL/data_modelo.csv",sep=";")
base_total_max = pd.read_csv("/content/drive/MyDrive/Proyecto_ADL/equipos.csv",sep=";")

In [4]:
X_base_model = new_df3.drop(columns=['team', 'team2', 'results', 'neutral_location'])
Y_model = new_df3['results']
Y_model

0        3
1        1
2        2
3        1
4        2
        ..
15654    1
15655    3
15656    3
15657    3
15658    1
Name: results, Length: 15659, dtype: int64

In [5]:
X_base_model.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15659 entries, 0 to 15658
Data columns (total 28 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   team1_rank  15659 non-null  float64
 1   team2_rank  15659 non-null  float64
 2   team1_gk    15659 non-null  float64
 3   team2_gk    15659 non-null  float64
 4   team1_de    15659 non-null  float64
 5   team1_of    15659 non-null  float64
 6   team1_mi    15659 non-null  float64
 7   team2_de    15659 non-null  float64
 8   team2_of    15659 non-null  float64
 9   team2_mi    15659 non-null  float64
 10  official_t  15659 non-null  int64  
 11  team1_af    15659 non-null  int64  
 12  team1_as    15659 non-null  int64  
 13  team1_eu    15659 non-null  int64  
 14  team1_na    15659 non-null  int64  
 15  team1_oc    15659 non-null  int64  
 16  team1_sa    15659 non-null  int64  
 17  team2_af    15659 non-null  int64  
 18  team2_as    15659 non-null  int64  
 19  team2_eu    15659 non-nul

In [6]:
X_base_model['official_t'] = X_base_model['official_t'].astype(float)
X_base_model['team1_af'] = X_base_model['team1_af'].astype(float)
X_base_model['team1_as'] = X_base_model['team1_as'].astype(float)
X_base_model['team1_eu'] = X_base_model['team1_eu'].astype(float)
X_base_model['team1_na'] = X_base_model['team1_na'].astype(float)
X_base_model['team1_oc'] = X_base_model['team1_oc'].astype(float)
X_base_model['team1_sa'] = X_base_model['team1_sa'].astype(float)
X_base_model['team2_af'] = X_base_model['team2_af'].astype(float)
X_base_model['team2_as'] = X_base_model['team2_as'].astype(float)
X_base_model['team2_eu'] = X_base_model['team2_eu'].astype(float)
X_base_model['team2_na'] = X_base_model['team2_na'].astype(float)
X_base_model['team2_oc'] = X_base_model['team2_oc'].astype(float)
X_base_model['team2_sa'] = X_base_model['team2_sa'].astype(float)
X_base_model.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15659 entries, 0 to 15658
Data columns (total 28 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   team1_rank  15659 non-null  float64
 1   team2_rank  15659 non-null  float64
 2   team1_gk    15659 non-null  float64
 3   team2_gk    15659 non-null  float64
 4   team1_de    15659 non-null  float64
 5   team1_of    15659 non-null  float64
 6   team1_mi    15659 non-null  float64
 7   team2_de    15659 non-null  float64
 8   team2_of    15659 non-null  float64
 9   team2_mi    15659 non-null  float64
 10  official_t  15659 non-null  float64
 11  team1_af    15659 non-null  float64
 12  team1_as    15659 non-null  float64
 13  team1_eu    15659 non-null  float64
 14  team1_na    15659 non-null  float64
 15  team1_oc    15659 non-null  float64
 16  team1_sa    15659 non-null  float64
 17  team2_af    15659 non-null  float64
 18  team2_as    15659 non-null  float64
 19  team2_eu    15659 non-nul

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X_base_model, Y_model, test_size=0.2, random_state=2022)

In [8]:
y_dict = [{'':str(x)} for x in y_train.tolist()]
y_dict_test = [{'':str(x)} for x in y_test.tolist()]
vec = DictVectorizer(sparse=False, dtype=int)
M_train = vec.fit_transform(y_dict)
M_test = vec.fit_transform(y_dict_test)

In [9]:
def get_model(meta, activation, n_layers, hidden_layer_1, hidden_layer_2 = None, hidden_layer_3 = None, hidden_layer_4 = None):
    # note that meta is a special argument that will be
    # handed a dict containing input metadata
    n_features_in_ = meta["n_features_in_"]
    X_shape_ = meta["X_shape_"]
    dims = [hidden_layer_1, hidden_layer_2, hidden_layer_3, hidden_layer_4]

    model = Sequential(name="Red_CV")
    model.add(Dense(n_features_in_, input_shape=X_shape_[1:]))

    for i in range(n_layers):
        model.add(Dense(dims[i], activation=activation))
    
    model.add(Dense(3, activation='softmax'))
    return model

In [10]:
clf = KerasClassifier(
    get_model,
    loss="categorical_crossentropy",
    n_layers=3,
    hidden_layer_1=30, 
    hidden_layer_2=10,
    hidden_layer_3=5,
    metrics=["accuracy"],
    epochs=10,
    activation="relu"
)


KerasClassifier(
	model=<function get_model at 0x7fdb639973b0>
	build_fn=None
	warm_start=False
	random_state=None
	optimizer=rmsprop
	loss=categorical_crossentropy
	metrics=['accuracy']
	batch_size=None
	validation_batch_size=None
	verbose=1
	callbacks=None
	validation_split=0.0
	shuffle=True
	run_eagerly=False
	epochs=10
	n_layers=3
	hidden_layer_1=30
	hidden_layer_2=10
	hidden_layer_3=5
	activation=relu
	class_weight=None
)

In [11]:
ct = ColumnTransformer([
       ('encoder', OneHotEncoder(handle_unknown='ignore'),
       make_column_selector(dtype_include=np.object)),
       ], remainder='passthrough')
estimators = [
              ('encoder', ct),
              ('clf', clf),
              ]

pipe = Pipeline(estimators)

In [150]:
pipe.fit(X_train, M_train)

Epoch 1/10
392/392 [==============================] - 2s 2ms/step - loss: 0.9527 - accuracy: 0.5688
Epoch 2/10
392/392 [==============================] - 1s 2ms/step - loss: 0.9169 - accuracy: 0.5811
Epoch 3/10
392/392 [==============================] - 1s 2ms/step - loss: 0.9083 - accuracy: 0.5867
Epoch 4/10
392/392 [==============================] - 1s 2ms/step - loss: 0.9046 - accuracy: 0.5855
Epoch 5/10
392/392 [==============================] - 1s 2ms/step - loss: 0.9008 - accuracy: 0.5859
Epoch 6/10
392/392 [==============================] - 1s 2ms/step - loss: 0.8988 - accuracy: 0.5873
Epoch 7/10
392/392 [==============================] - 1s 2ms/step - loss: 0.8977 - accuracy: 0.5875
Epoch 8/10
392/392 [==============================] - 1s 2ms/step - loss: 0.8961 - accuracy: 0.5885
Epoch 9/10
392/392 [==============================] - 1s 2ms/step - loss: 0.8950 - accuracy: 0.5890
Epoch 10/10
392/392 [==============================] - 1s 3ms/step - loss: 0.8934 - accuracy: 0.5888

Pipeline(steps=[('encoder',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('encoder',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7fdb63458f50>)])),
                ('clf',
                 KerasClassifier(activation='relu', epochs=10, hidden_layer_1=30, hidden_layer_2=10, hidden_layer_3=5, loss='categorical_crossentropy', metrics=['accuracy'], model=<function get_model at 0x7fdb639973b0>, n_layers=3))])

In [151]:
y_pred = pipe.predict(X_train)
y_pred
print(classification_report(M_train, y_pred))

392/392 [==============================] - 1s 2ms/step
              precision    recall  f1-score   support

           0       0.60      0.88      0.71      6096
           1       0.33      0.03      0.05      2799
           2       0.58      0.53      0.56      3632

   micro avg       0.59      0.59      0.59     12527
   macro avg       0.51      0.48      0.44     12527
weighted avg       0.54      0.59      0.52     12527
 samples avg       0.59      0.59      0.59     12527



In [14]:
with tf.device('/device:GPU:0'): 
    params = {
              "clf__hidden_layer_1": [8, 100, 10, 5],
              "clf__hidden_layer_2": [8, 100, 10, 5],
              "clf__hidden_layer_3": [8, 100, 10, 5],
              "clf__n_layers": [1,2,3,4],
              "clf__activation": ["relu", "linear", "sigmoid"],
              "clf__optimizer": ["adam", "sgd"],
              "clf__optimizer__learning_rate": [0.0001, 0.001, 0.1],
              "clf__epochs": [10, 30, 20],
             }
    grid = RandomizedSearchCV(pipe, params, cv=3, scoring='neg_log_loss', n_iter=10, random_state=2022)

In [15]:
with tf.device('/device:GPU:0'): 
  grid.fit(X_train, y_train)

Epoch 1/10
261/261 [==============================] - 1s 2ms/step - loss: 0.9344 - accuracy: 0.5684
Epoch 2/10
261/261 [==============================] - 1s 2ms/step - loss: 0.9131 - accuracy: 0.5821
Epoch 3/10
261/261 [==============================] - 0s 2ms/step - loss: 0.9087 - accuracy: 0.5826
Epoch 4/10
261/261 [==============================] - 0s 2ms/step - loss: 0.9072 - accuracy: 0.5853
Epoch 5/10
261/261 [==============================] - 1s 2ms/step - loss: 0.9041 - accuracy: 0.5862
Epoch 6/10
261/261 [==============================] - 0s 2ms/step - loss: 0.9041 - accuracy: 0.5852
Epoch 7/10
261/261 [==============================] - 0s 2ms/step - loss: 0.9033 - accuracy: 0.5847
Epoch 8/10
261/261 [==============================] - 0s 2ms/step - loss: 0.9030 - accuracy: 0.5872
Epoch 9/10
261/261 [==============================] - 0s 2ms/step - loss: 0.9016 - accuracy: 0.5841
Epoch 10/10
131/131 [==============================] - 0s 1ms/step
Epoch 1/10
261/261 [=============

In [107]:
grid.best_params_

{'clf__optimizer__learning_rate': 0.1,
 'clf__optimizer': 'sgd',
 'clf__n_layers': 1,
 'clf__hidden_layer_3': 5,
 'clf__hidden_layer_2': 100,
 'clf__hidden_layer_1': 8,
 'clf__epochs': 10,
 'clf__activation': 'relu'}

In [146]:
def get_model(meta, activation = 'relu', n_layers = 1, hidden_layer_1 = 8, hidden_layer_2 = 100, hidden_layer_3 = 5):
    # note that meta is a special argument that will be
    # handed a dict containing input metadata
    n_features_in_ = X_train.shape[1]
    X_shape_ = X_train.shape[1]
    activation = 'relu'
    n_layers = 1
    hidden_layer_1 = 8
    hidden_layer_2 = 100
    hidden_layer_3 = 5
    dims = [hidden_layer_1, hidden_layer_2, hidden_layer_3]

    model = Sequential(name="Red_CV")
    model.add(Input(shape=(X_train.shape[1]),name='Capa_Entrada'))

    for i in range(n_layers):
        model.add(Dense(dims[i], activation=activation))
    
    model.add(Dense(3, activation='softmax'))
    model.compile(loss='categorical_crossentropy',
                optimizer=tf.keras.optimizers.SGD(learning_rate=0.1),
                metrics=["accuracy",tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
    return model

In [147]:
tuner = kt.RandomSearch(
    get_model,
    objective="val_accuracy",
    project_name="project_adl",
    seed=123,
    max_trials=1)

In [153]:
with tf.device('/device:GPU:0'): 
    tuner.search(X_train,
                 M_train,
             validation_data=(X_test, M_test),
             epochs=10)
    
best_model = tuner.get_best_models()[0]

Trial 1 Complete [00h 00m 13s]
val_accuracy: 0.5766283273696899

Best val_accuracy So Far: 0.5766283273696899
Total elapsed time: 00h 00m 13s


In [154]:
best_model.save('best_model_ADL.h5')
best_model.summary()

Model: "Red_CV"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 8)                 232       
                                                                 
 dense_1 (Dense)             (None, 3)                 27        
                                                                 
Total params: 259
Trainable params: 259
Non-trainable params: 0
_________________________________________________________________


In [89]:
model = Sequential(name="RedNeuronal")
model.add(Input(shape=(X_train.shape[1]),name='Capa_Entrada'))
model.add(Dense(8,activation='relu',name='Capa_Densa_1'))
model.add(Dense(100,activation='relu',name='Capa_Densa_2'))
model.add(Dense(5,activation='relu',name='Capa_Densa_3'))
model.add(Dense(3,activation='softmax',name='Capa_Respuesta'))
model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.SGD(learning_rate=0.1) ,metrics=[tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
model.summary()

history = model.fit(X_train, 
        M_train,           
        validation_data=([X_test, M_test]),
        epochs=10
    )

Model: "RedNeuronal"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Capa_Densa_1 (Dense)        (None, 8)                 232       
                                                                 
 Capa_Densa_2 (Dense)        (None, 100)               900       
                                                                 
 Capa_Densa_3 (Dense)        (None, 5)                 505       
                                                                 
 Capa_Respuesta (Dense)      (None, 3)                 18        
                                                                 
Total params: 1,655
Trainable params: 1,655
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
392/392 [==============================] - 2s 4ms/step - loss: 0.9571 - precision_9: 0.6391 - recall_9: 0.3879 - val_loss: 0.9392 - val_precision_9: 0.6441 - val_recall_9: 0.

In [96]:
y_pred = best_model.predict(X_train)
y_pred
print(classification_report(y_train, y_pred))

392/392 [==============================] - 1s 3ms/step
              precision    recall  f1-score   support

           1       0.60      0.89      0.71      6096
           2       0.30      0.00      0.01      2799
           3       0.58      0.54      0.56      3632

    accuracy                           0.59     12527
   macro avg       0.49      0.48      0.43     12527
weighted avg       0.52      0.59      0.51     12527



In [156]:
val_pred = np.argmax(best_model.predict(X_train), axis=-1) + 1
val_pred
print(classification_report(y_train, val_pred))

392/392 [==============================] - 1s 1ms/step
              precision    recall  f1-score   support

           1       0.60      0.86      0.71      6096
           2       0.21      0.00      0.00      2799
           3       0.55      0.58      0.56      3632

    accuracy                           0.59     12527
   macro avg       0.46      0.48      0.43     12527
weighted avg       0.50      0.59      0.51     12527



In [24]:
y_pred = best_model.predict(X_test)
y_pred
print(classification_report(y_test, y_pred))

98/98 [==============================] - 0s 1ms/step
              precision    recall  f1-score   support

           1       0.58      0.88      0.70      1498
           2       0.67      0.01      0.02       748
           3       0.55      0.53      0.54       886

    accuracy                           0.57      3132
   macro avg       0.60      0.47      0.42      3132
weighted avg       0.59      0.57      0.49      3132



In [157]:
val_pred = np.argmax(best_model.predict(X_test), axis=-1) + 1
val_pred
print(classification_report(y_test, val_pred))

98/98 [==============================] - 0s 4ms/step
              precision    recall  f1-score   support

           1       0.59      0.87      0.70      1498
           2       0.67      0.00      0.01       748
           3       0.54      0.57      0.55       886

    accuracy                           0.58      3132
   macro avg       0.60      0.48      0.42      3132
weighted avg       0.59      0.58      0.49      3132



In [25]:
base_total_max['team']

0         Afghanistan
1             Albania
2             Algeria
3      American Samoa
4             Andorra
            ...      
207           Vietnam
208             Wales
209             Yemen
210            Zambia
211          Zimbabwe
Name: team, Length: 212, dtype: object

In [186]:
def obtain_new(equipo1, equipo2):
    base1 = base_total_max[base_total_max['team'] == equipo1]
    base1 = base1.drop(columns=['date'])
    base2 = base_total_max[base_total_max['team'] == equipo2]
    base2 = base2.drop(columns=['date'])
    base2 = base2.rename(columns={"team": "team2",
                                                  "team1_rank": "team2_rank",
                                                  "team1_gk": "team2_gk",
                                                  "team1_de": "team2_de",
                                                  "team1_of": "team2_of",
                                                  "team1_mi": "team2_mi", 
                                                  "team1_af": "team2_af",
                                                  "team1_as": "team2_as",
                                                  "team1_eu": "team2_eu",
                                                  "team1_na": "team2_na",
                                                  "team1_oc": "team2_oc",
                                                  "team1_sa": "team2_sa"})
    base1 = base1.drop(columns=['team'])
    base2 = base2.drop(columns=['team2'])
    base_total = pd.concat([base1.reset_index(drop=True), base2.reset_index(drop=True)], axis=1)
    base_total['official_t'] = 1
    base_total['dif_rank'] = base_total['team1_rank'] - base_total['team2_rank']
    base_total['dif_gf'] = base_total['team1_gk'] - base_total['team2_gk']
    base_total['dif_de'] = base_total['team1_de'] - base_total['team2_de']
    base_total['dif_of'] = base_total['team1_of'] - base_total['team2_of']
    base_total['dif_mi'] = base_total['team1_mi'] - base_total['team2_mi']
    prediccion1 = best_model.predict(base_total)[0]
    base1 = base_total_max[base_total_max['team'] == equipo2]
    base1 = base1.drop(columns=['date'])
    base2 = base_total_max[base_total_max['team'] == equipo1]
    base2 = base2.drop(columns=['date'])
    base2 = base2.rename(columns={"team": "team2",
                                                  "team1_rank": "team2_rank",
                                                  "team1_gk": "team2_gk",
                                                  "team1_de": "team2_de",
                                                  "team1_of": "team2_of",
                                                  "team1_mi": "team2_mi", 
                                                  "team1_af": "team2_af",
                                                  "team1_as": "team2_as",
                                                  "team1_eu": "team2_eu",
                                                  "team1_na": "team2_na",
                                                  "team1_oc": "team2_oc",
                                                  "team1_sa": "team2_sa"})
    base1 = base1.drop(columns=['team'])
    base2 = base2.drop(columns=['team2'])
    base_total = pd.concat([base1.reset_index(drop=True), base2.reset_index(drop=True)], axis=1)
    base_total['official_t'] = 1
    base_total['dif_rank'] = base_total['team1_rank'] - base_total['team2_rank']
    base_total['dif_gf'] = base_total['team1_gk'] - base_total['team2_gk']
    base_total['dif_de'] = base_total['team1_de'] - base_total['team2_de']
    base_total['dif_of'] = base_total['team1_of'] - base_total['team2_of']
    base_total['dif_mi'] = base_total['team1_mi'] - base_total['team2_mi']
    prediccion2 = best_model.predict(base_total)[0]
    prediccion_f1 = prediccion1[0] + prediccion2[2]
    prediccion_f2 = prediccion1[1] + prediccion2[1]
    prediccion_f3 = prediccion1[2] + prediccion2[0]
    prediccion_final = [prediccion_f1/2, prediccion_f2/2, prediccion_f3/2]
    return prediccion_final

In [190]:
obtain_new('Portugal', 'Uruguay')

1/1 [==============================] - 0s 32ms/step


[0.40551379323005676, 0.20496535301208496, 0.3895208239555359]

In [188]:
obtain_new('Brazil', 'Qatar')

1/1 [==============================] - 0s 21ms/step


[0.6897290945053101, 0.1247374638915062, 0.18553343415260315]

In [30]:
obtain_new('Brazil', 'Qatar')

1/1 [==============================] - 0s 19ms/step


[0.7360963225364685, 0.1673298478126526, 0.09657388180494308]